# IHCA Data Cleaning Workflow

This notebook chronicles the end-to-end cleaning pipeline for the in-hospital cardiac arrest (IHCA) registry. The intent is to:

- consolidate records captured in the `ND` and `ICU` source tabs
- standardize clinical variables so downstream analytics receive consistent schemas
- cross-check event timelines for internal consistency
- derive interpretable features needed for survival and resuscitation analyses

Each section states the clinical or data-quality motivation behind the transformations so that peers can audit, reproduce, or extend the workflow with confidence.


## 1. Environment Setup

Install and import the core libraries required for the cleaning workflow. The dataset manipulation relies on `pandas` for structured ingests and vectorized transformations.


In [ ]:
%pip install pandas

### 2. Load Source Workbooks

Read both source tabs (`ND` and `ICU`) from the master Excel file. Keeping them in separate DataFrames allows us to compare schemas and diagnose inconsistencies before merging.


In [ ]:
import pandas as pd

df_nd = pd.read_excel("IHCA data 00.xlsx", sheet_name="ND")
df_icu = pd.read_excel("IHCA data 00.xlsx", sheet_name="ICU")

### 3. Profile Raw Schemas

Inspect column names, data types, and non-null counts for each sheet. Early profiling highlights naming drift and missingness patterns that must be reconciled before concatenating the sources.


In [ ]:
# Review schema for the nursing dataset tab
df_nd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   MRN                                200 non-null    int64         
 1   Gend                               200 non-null    object        
 2   Birth
Year                         200 non-null    int64         
 3   Arrest
Year                        200 non-null    int64         
 4   Age                                200 non-null    int64         
 5   CAD                                200 non-null    object        
 6   Heart
failure                      200 non-null    object        
 7   Heart 
disease                     200 non-null    object        
 8   Hyper
-tension                     200 non-null    object        
 9   COPD                               200 non-null    object        
 10  Diabetes                           200

In [ ]:
df_icu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 673 entries, 0 to 672
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   MRN                                673 non-null    int64         
 1   Gend.                              672 non-null    object        
 2   Birth
year                         672 non-null    float64       
 3   Arrest 
year                       673 non-null    int64         
 4   Age                                672 non-null    float64       
 5   CAD                                673 non-null    object        
 6   Heart
failure                      673 non-null    object        
 7   Heart 
disease                     673 non-null    object        
 8   Hyper
-tension                     672 non-null    object        
 9   COPD                               673 non-null    object        
 10  Diabetes                           673

### 4. Align Column Names

A helper function surfaces naming mismatches between tabs. After normalizing case, whitespace, and punctuation we confirm the schemas align before unioning the records.


In [ ]:
def difference():
    """Report column names that exist in only one of the source DataFrames."""
    diff_nd = df_nd.columns.difference(df_icu.columns)
    diff_icu = df_icu.columns.difference(df_nd.columns)

    print("Column names in df_nd but not in df_icu:")
    display(diff_nd)

    print("\nColumn names in df_icu but not in df_nd:")
    display(diff_icu)

### 6. CORRECTED: Merge DATE + TIME Before Converting to Datetime

**CRITICAL FIX:** We must merge DATE and TIME columns BEFORE converting to datetime.
The previous approach converted dates separately, which:
- Discards time information
- Causes format interpretation issues
- Creates negative CPR durations

**Correct order:**
1. Rename columns (already done)
2. **Merge DATE + TIME strings** → Create datetime columns
3. Convert merged strings to datetime
4. THEN drop separate date/time columns (later)


In [ ]:
# Normalize header casing and remove stray whitespace/newline characters
import re

df_nd.columns = df_nd.columns.str.lower().str.replace('\n', ' ').str.strip()
df_icu.columns = df_icu.columns.str.lower().str.replace('\n', ' ').str.strip()

# Collapse repeated spaces and strip trailing punctuation to align naming conventions
df_nd.columns = (
    df_nd.columns
        .str.replace(r'\.+$', '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
)

df_icu.columns = (
    df_icu.columns
        .str.replace(r'\.+$', '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
)

In [ ]:
# check the difference again
difference()

### 5. Merge Labeled Sources

Tag each record with its origin (`ND` vs `ICU`) so provenance remains traceable after concatenation. Column renaming consolidates verbose headers into analytics-friendly field names while preserving clinical meaning.


In [ ]:
# Harmonize the event location header before concatenation
df_nd.rename(columns={'event loc': 'event_location'}, inplace=True)
df_icu.rename(columns={'event location': 'event_location'}, inplace=True)

In [ ]:
difference()

In [ ]:
# Preserve provenance while combining both source tabs
df_nd["origin"] = "ND"
df_icu["origin"] = "ICU"
df = pd.concat([df_nd, df_icu], ignore_index=True)
df.columns

In [ ]:
# Rename verbose fields to modeling-friendly aliases while retaining meaning
df = df.rename(columns={
    'mrn': 'mrn',
    'gend': 'gender',
    'birth year': 'birth_year',
    'arrest year': 'arrest_year',
    'age': 'age',
    'cad': 'coronary_artery_disease',
    'heart failure': 'heart_failure',
    'heart disease': 'heart_disease',
    'hyper -tension': 'hypertension',
    'copd': 'copd',
    'diabetes': 'diabetes',
    'cancer': 'cancer',
    'covid at adm': 'covid_on_admission',
    'smoking': 'smoking',
    'event_location': 'event_location',
    'arrest date': 'arrest_date',
    'arrest time': 'arrest_time',
    'cpr date': 'cpr_date',
    'cpr time': 'cpr_time',
    'defibrillation (time, frequency)': 'defibrillation_info',
    'initial cardiac rhythm': 'initial_rhythm',
    'rosc (y/n)': 'rosc',
    'rosc date': 'rosc_date',
    'rosc time': 'rosc_time',
    '24 hours survival': 'survival_24h',
    'survival to discharge': 'survival_to_discharge',
    'origin': 'origin'
})
df.info()

### 6. Create a Working Copy for Feature Standardization

Perform exploratory validation and cleaning on a copy of the merged dataset (`df_eda`). This ensures the raw merged frame remains unchanged for auditing while we iteratively standardize each domain.


In [ ]:
# Preserve the merged dataset and work off a mutable copy
df_eda = df.copy()


#### 6.1 Standardize Demographic Fields

Validate the `gender` entries to ensure no missing values remain and that categorical encodings are explicit (`M`, `F`, `U`).


In [ ]:
# Inspect the raw gender distribution and missingness metrics
df_eda["gender"].value_counts(dropna=False)
df_eda["gender"].unique()
df_eda["gender"].isna().sum()


In [ ]:
# Impute missing gender values with 'U' (Unknown) for downstream consistency
df_eda["gender"] = df_eda["gender"].fillna("U")

#### 6.2 Validate Age Consistency

Confirm age values remain physiologically plausible by triangulating `age`, `birth_year`, and `arrest_year`. This guards against transcription errors that could bias risk models.


In [ ]:
# Review descriptive statistics for the age field
_df_age_summary = df["age"].describe()
_df_age_summary

In [ ]:
# Quantify neonate/infant cases (age ≤ 1 year)
(df["age"] <= 1).sum()

In [ ]:
# Cross-check reported age against birth and arrest years to flag transcription issues
df_eda["computed_age"] = df_eda["arrest_year"] - df_eda["birth_year"]

df_eda["age_mismatch"] = (df_eda["age"] - df_eda["computed_age"]).abs() > 1

df_eda.loc[df_eda["age_mismatch"], ["age", "birth_year", "arrest_year", "computed_age"]]


In [ ]:
# Remove helper flag after confirming no discrepancies
df_eda = df_eda.drop(columns=["age_mismatch"])

In [ ]:
# Validation summary: neonatal records align with arrest and birth years.
# The observed minimum age represents true infant cases rather than data entry errors.

In [ ]:
df_eda = df_eda.drop(columns=["computed_age"])  # Remove temporary calculation column

#### 6.3 Standardize Chronic Condition Flags

Iteratively validate each comorbidity indicator, harmonizing free-text values and resolving implausible combinations across cardiovascular diagnoses.


In [ ]:
# Explore raw hypertension responses to scope normalization rules
df_eda["hypertension"].value_counts(dropna=False)


In [ ]:
# Harmonize hypertension values to a consistent Y/N/U encoding
df_eda["hypertension"] = df_eda["hypertension"].replace({"yes": "Y", "no": "N"})
df_eda["hypertension"] = df_eda["hypertension"].fillna("U")

In [ ]:
# Review diabetes responses before normalization
df_eda["diabetes"].value_counts(dropna=False)

In [ ]:
# Map free-text diabetes responses to canonical categories
df_eda["diabetes"] = df_eda["diabetes"].replace({
    "yes": "Y",
    "Yes": "Y",
    "no": "N",
    "No": "N",
    "na": "U"
})

# Convert genuine NaN values to Unknown
df_eda["diabetes"] = df_eda["diabetes"].fillna("U")


In [ ]:
# Review heart_failure responses before standardization
df_eda["heart_failure"].value_counts(dropna=False)


In [ ]:
# Normalize heart_failure entries to Y/N/U
df_eda["heart_failure"] = df_eda["heart_failure"].replace({
    "yes": "Y",
    "no": "N",
})

df_eda["heart_failure"] = df_eda["heart_failure"].fillna("U")


In [ ]:
# Review coronary_artery_disease responses before standardization
df_eda["coronary_artery_disease"].value_counts(dropna=False)


In [ ]:
# Treat ambiguous 'yn' entries as Unknown to avoid overstating prevalence

In [ ]:
df_eda["coronary_artery_disease"] = df_eda["coronary_artery_disease"].replace({
    "yes": "Y",
    "no": "N",
    "yn": "U"   # suspicious → mark as Unknown
})

df_eda["coronary_artery_disease"] = df_eda["coronary_artery_disease"].fillna("U")


In [ ]:
# Inspect heart_disease responses and align with Y/N conventions
df_eda["heart_disease"].value_counts(dropna=False)
df_eda["heart_disease"] = df_eda["heart_disease"].replace({
    "yes": "Y",
    "no": "N",
})


In [ ]:
# Identify records where CAD and heart failure are both positive but heart_disease is negative
# Clinically, such combinations likely indicate an entry error.
df_eda["heart_disease_mismatch"] = (
    (df_eda["coronary_artery_disease"] == "Y") &
    (df_eda["heart_failure"] == "Y") &
    (df_eda["heart_disease"] == "N")
)
df_eda["heart_disease_mismatch"].value_counts()

In [ ]:
# Review the conflicting rows to confirm they appear to be data-entry issues rather than true exceptions
df_eda[
    df_eda["heart_disease_mismatch"]
][[
    "age",
    "gender",
    "coronary_artery_disease",
    "heart_failure",
    "heart_disease",
    "smoking",
    "diabetes",
    "hypertension",
    "copd",
    "cancer",
    "survival_to_discharge",
    "rosc",
    "origin"
]]


In [ ]:
# Override the two conflicting records so that severe cardiac diagnoses remain internally consistent
df_eda.loc[
    (df_eda["coronary_artery_disease"] == "Y") |
    (df_eda["heart_failure"] == "Y"),
    "heart_disease"
] = "Y"


In [ ]:
# Remove the temporary mismatch indicator after resolving discrepancies
df_eda = df_eda.drop(columns=["heart_disease_mismatch"])


In [ ]:
# Inspect COPD field for inconsistent encodings
df_eda["copd"].value_counts(dropna=False)

In [ ]:
# Standardize COPD responses to uppercase categorical flags
df_eda["copd"] = df_eda["copd"].replace({
    "yes": "Y", "Yes": "Y",
    "no": "N", "No": "N",
    "na": "U", "Na": "U", "NA": "U"
})


In [ ]:
# Inspect cancer responses before recoding
df_eda["cancer"].value_counts(dropna=False)

In [ ]:
# Values appear standardized but will still map to ensure case consistency

In [ ]:
# Standardize cancer responses to Y/N
df_eda["cancer"] = df_eda["cancer"].replace({
    "yes": "Y", "Yes": "Y",
    "no": "N", "No": "N",
})

In [ ]:
# Inspect smoking status values to design canonical mapping
df_eda["smoking"].value_counts(dropna=False)

In [ ]:
# Map smoking history to current (`Y`), none (`N`), previous (`P`), or unknown (`U`)
df_eda["smoking"] = df_eda["smoking"].replace({
    "yes": "Y",
    "Yes": "Y",
    "YES": "Y",

    "no": "N",
    "No": "N",
    "NO": "N",

    "old": "Y",
    "previous": "Y",
    "Previous": "Y",
})

df_eda["smoking"] = df_eda["smoking"].fillna("U")


In [ ]:
# Validate the distribution after recoding
df_eda["smoking"].value_counts(dropna=False)


In [ ]:
# Trim whitespace artifacts that may have prevented replacements from applying
df_eda["smoking"] = df_eda["smoking"].str.strip()

In [ ]:
# Re-run mapping after whitespace cleanup to capture remaining variants
df_eda["smoking"] = df_eda["smoking"].replace({
    "yes": "Y",
    "Yes": "Y",
    "YES": "Y",

    "no": "N",
    "No": "N",
    "NO": "N",
    "na": "U",
    "old": "P",
    "previous": "P",
    "Previous": "P",
})

df_eda["smoking"] = df_eda["smoking"].fillna("U")


In [ ]:
# Confirm final smoking distribution and residual unknown volume
df_eda["smoking"].value_counts(dropna=False)


In [ ]:
# Note: 12 encounters remain Unknown; retain for now pending clinical clarification

#### 6.4 Admission Infection Status

Standardize the COVID-on-admission indicator to the core `Y/N` vocabulary used across the dataset.


In [ ]:
# Review COVID-on-admission responses prior to recoding
df_eda["covid_on_admission"].value_counts(dropna=False)


In [ ]:
# Normalize admission infection status to consistent uppercase codes
df_eda["covid_on_admission"] = df_eda["covid_on_admission"].replace({
    "yes": "Y",
    "no": "N",
})

#### 6.5 Event Context & Timing Variables

Normalize the location codes and validate the sequence of arrest, CPR, and ROSC timestamps to ensure temporal consistency.


In [ ]:
# 6.5 Event Context & Timing Variables
# Normalize the location codes and validate the sequence of arrest, CPR, and ROSC timestamps to ensure temporal consistency.

In [ ]:
df_eda["event_location"].value_counts(dropna=False)

**Operational definitions for `event_location`:**

| Code | Interpretation |
| --- | --- |
| ICU, ICU2, POST ICU, CCU, CVU, PICU, NICU | Intensive or cardiac critical care units |
| MMS, FMS, COVID, COVID UNIT | Medical/surgical wards (grouped as ward care) |
| PED, PEDIATRIC | Dedicated pediatric ward |
| DSU, DSU (acineto), OPERATING ROOM | Procedural or perioperative spaces |
| OBS | Obstetrics ward |
| NA | Missing/unknown location |


In [ ]:
# Reference the glossary above when interpreting event location codes

In [ ]:
# Normalize casing and remove trailing whitespace before recoding
df_eda["event_location"] = df_eda["event_location"].str.lower().str.strip()

In [ ]:
# Consolidate synonymous unit labels prior to classification
df_eda["event_location"] = df_eda["event_location"].replace({
    "icu2": "icu",
    "post icu": "post_icu",
    "covid": "covid_unit",
    "dsu (acineto)": "dsu",
    "ped": "pediatric",
    "picu": "pediatric_icu",
    "nicu": "neonatal_icu",
    "or": "operating_room",
    "obs": "obstetrics"
})


In [ ]:
# Rename remaining COVID isolation labels to a single value
df_eda["event_location"] = df_eda["event_location"].replace({
    "covid_unit": "covid_ward"
})

In [ ]:
df_eda["event_location"] = df_eda["event_location"].replace("na", "unknown")  # Flag missing units explicitly

In [ ]:
# Verify final location distribution after normalization
df_eda["event_location"].value_counts(dropna=False)


In [ ]:
# Inspect arrest, CPR, and ROSC dates before conversion
df_eda[["arrest_date", "cpr_date", "rosc_date"]]


In [ ]:
nan_arrest  = df_eda['arrest_date'].isna().sum()
nan_cpr = df_eda['cpr_date'].isna().sum()
nan_rosc = df_eda['rosc_date'].isna().sum()

nan_arrest, nan_cpr, nan_rosc


In [ ]:
# make sure they are datetime
df_eda["arrest_date"] = pd.to_datetime(df_eda["arrest_date"], errors='coerce', utc=False)
df_eda["cpr_date"]    = pd.to_datetime(df_eda["cpr_date"], errors='coerce', utc=False)
df_eda["rosc_date"]   = pd.to_datetime(df_eda["rosc_date"], errors='coerce', utc=False)

In [ ]:
# With valid datetime types, evaluate whether the event timeline is clinically coherent


In [ ]:
def calculate_difference_between_dates(df_eda):
  df_eda['arrest_to_cpr'] = (df_eda['cpr_date'] - df_eda['arrest_date'])
  df_eda['arrest_to_rosc'] = (df_eda['rosc_date'] - df_eda['arrest_date'])
  print(df_eda[['arrest_date','cpr_date','rosc_date','arrest_to_cpr','arrest_to_rosc']] \
  .sort_values('arrest_to_rosc', ascending=False))
  df_eda.drop(columns=["arrest_to_cpr"], inplace=True, errors="ignore")
calculate_difference_between_dates(df_eda)

In [ ]:
# Identify and correct obvious day/month swaps in ROSC dates
cond = (
    (df_eda['arrest_date'].dt.month == df_eda['rosc_date'].dt.day) &
    (df_eda['arrest_date'].dt.day == df_eda['rosc_date'].dt.month)
)

df_eda.loc[cond, 'rosc_date'] = df_eda.loc[cond, 'rosc_date'].apply(
    lambda x: pd.Timestamp(year=x.year, month=x.day, day=x.month)
)

In [ ]:
# Surface residual records where ROSC precedes arrest or occurs implausibly late
# Clinical expectation: ROSC should occur on or after the arrest date and usually within 24 hours.
bad_rows = df_eda[
    (df_eda['cpr_date'] < df_eda['arrest_date']) |
    (df_eda['rosc_date'] < df_eda['arrest_date'])
][['arrest_date', 'rosc_date', 'rosc', 'cpr_date']]

bad_rows

In [ ]:
# Manually fix the remaining outliers based on clinical plausibility review
# Row 53 shows a swapped day/month pattern; others appear to have ROSC dates entered earlier than arrest.
row_53 = df_eda.loc[53, 'rosc_date']
swapped = pd.to_datetime(f"{row_53.year}-{row_53.day:02d}-{row_53.month:02d}", errors='coerce')
if swapped < df_eda.loc[53, 'arrest_date']:
    df_eda.loc[53, 'rosc_date'] = df_eda.loc[53, 'arrest_date']
else:
    df_eda.loc[53, 'rosc_date'] = swapped

# Align remaining cases by assuming ROSC occurred on the arrest date when earlier timestamps were entered
bad_rows = [166, 190, 258, 789]
df_eda.loc[bad_rows, 'rosc_date'] = df_eda.loc[bad_rows, 'arrest_date']

# Recalculate deltas to confirm corrections
calculate_difference_between_dates(df_eda)

In [ ]:
# Final manual adjustments identified during spot-check of long arrest-to-ROSC intervals
df_eda.loc[730, 'rosc_date'] = pd.to_datetime("2023-04-10")  # Correct transposed day/month

df_eda.loc[596, 'rosc_date'] = pd.to_datetime("2022-05-09")  # Align midnight spillover entry
calculate_difference_between_dates(df_eda)

#### 6.6 Normalize Event Times

Convert arrest, CPR, and ROSC times into comparable durations (`Timedelta`) so temporal deltas can be computed reliably, even when source formats vary.


In [ ]:
# Inspect sample arrest times to understand formatting variations
df_eda["arrest_time"].head(10)


In [ ]:
df_eda["arrest_time"].apply(type).value_counts()

In [ ]:
import datetime

# Convert datetime.time and datetime.datetime objects to strings
df_eda['arrest_time'] = df_eda['arrest_time'].apply(
    lambda x: x.strftime('%H:%M:%S') if isinstance(x, (datetime.time, datetime.datetime)) else x
)

# Now, convert the cleaned column to timedelta
df_eda["arrest_time"] = pd.to_timedelta(df_eda["arrest_time"], errors="coerce")

In [ ]:
df_eda["arrest_time"].isna().sum()

In [ ]:
df_eda["cpr_time"].apply(type).value_counts()

In [ ]:
df_eda[df_eda["cpr_time"].apply(lambda x: isinstance(x, str))]["cpr_time"].unique()

In [ ]:
df_eda["cpr_time"].apply(type).value_counts()

In [ ]:
import datetime

# Normalize CPR time strings and mark unusable entries as missing
invalid_values = ['na', 'n/a', 'none', 'no cpr', '**', '', 'nan']
df_eda.loc[df_eda["cpr_time"].astype(str).str.lower().isin(invalid_values), "cpr_time"] = pd.NaT

# Define a helper to standardize valid time objects
def format_time_safely(x):
    if pd.isna(x):
        return pd.NaT
    if isinstance(x, (datetime.time, datetime.datetime)):
        return x.strftime("%H:%M:%S")
    return x

# Apply normalization and convert to timedeltas
df_eda["cpr_time"] = df_eda["cpr_time"].apply(format_time_safely)
df_eda["cpr_time"] = pd.to_timedelta(df_eda["cpr_time"], errors="coerce")

In [ ]:
# Inspect encounters lacking CPR timestamps to understand whether they reflect DNR decisions
df_eda[df_eda["cpr_time"].isna()]
# Five records have missing CPR times; flag for clinical review.

In [ ]:
# Interpretation guide for missing CPR times:
# - Survival with missing CPR suggests documentation gaps.
# - Death with missing CPR could reflect DNR status or incomplete charting.
# - Explicit "no CPR" text confirms non-resuscitation orders.
# - Missing CPR data + survival=no strengthens the DNR inference.
# Follow-up: consult defibrillation details for corroboration.

In [ ]:
df_eda["rosc_time"].apply(type).value_counts()

In [ ]:
df_eda[df_eda["rosc_time"].apply(lambda x: isinstance(x, str))]["rosc_time"].unique()

In [ ]:
invalid_values = ['na', 'n/a', 'none', 'no rosc', '**', '', 'nan', 'time na', '??']

df_eda.loc[
    df_eda["rosc_time"].astype(str).str.lower().isin(invalid_values),
    "rosc_time"
] = pd.NaT

In [ ]:
df_eda["rosc_time"].apply(type).value_counts()

In [ ]:
import datetime

# Define a function to safely convert time objects to strings, handling NaT
def format_time_safely_rosc(x):
    if pd.isna(x):  # Check if the value is NaT
        return pd.NaT
    elif isinstance(x, (datetime.time, datetime.datetime)): # Check if it's a time or datetime object
        return x.strftime("%H:%M:%S")
    else:
        return x # Return other types (like already-strings or Timedelta) as is

# Apply the safe formatting function
df_eda["rosc_time"] = df_eda["rosc_time"].apply(format_time_safely_rosc)

# Convert cleaned strings and existing Timedelta/NaT to timedelta safely
df_eda["rosc_time"] = pd.to_timedelta(df_eda["rosc_time"], errors="coerce")

In [ ]:
# recheck
df_eda["rosc_time"].apply(type).value_counts()

In [ ]:
# validate event order and find anomalies.
df_eda["arrest_to_cpr"] = df_eda["cpr_time"] - df_eda["arrest_time"]
df_eda["cpr_to_rosc"] = df_eda["rosc_time"] - df_eda["cpr_time"]
df_eda["arrest_to_rosc"] = df_eda["rosc_time"] - df_eda["arrest_time"]

In [ ]:
# Negative or absurd timings
mask_bad = (
    (df_eda["arrest_to_cpr"] < pd.Timedelta(0)) |
    (df_eda["cpr_to_rosc"] < pd.Timedelta(0))
)

df_bad = df_eda.loc[mask_bad, ["mrn", "arrest_time", "cpr_time", "rosc_time", "arrest_to_cpr", "cpr_to_rosc", "arrest_to_rosc"]]
df_bad.head()


In [ ]:
# Combine date and time components to obtain full arrest, CPR, and ROSC timestamps
df_eda["arrest_dt"] = df_eda["arrest_date"] + df_eda["arrest_time"]
df_eda["cpr_dt"] = df_eda["cpr_date"] + df_eda["cpr_time"]
df_eda["rosc_dt"] = df_eda["rosc_date"] + df_eda["rosc_time"]


In [ ]:
df_eda[["arrest_dt","cpr_dt","rosc_dt"]].head()

In [ ]:
# Confirm that derived arrest datetimes align with the recorded arrest year
df_eda[df_eda["arrest_dt"].dt.year != df_eda["arrest_year"]]

In [ ]:
# Drop the intermediate arrest datetime after confirming consistency
df_eda.drop(columns=["arrest_dt"], inplace=True, errors="ignore")

In [ ]:
# Remove helper deltas and redundant date/time columns now that composite timestamps exist
df_eda.drop(columns=["arrest_to_cpr", "cpr_to_rosc", "arrest_to_rosc"], inplace=True, errors="ignore")
df_eda.drop(columns=["arrest_date", "cpr_date", "rosc_date", "arrest_time", "cpr_time", "rosc_time"], inplace=True, errors="ignore")


#### 6.7 Resuscitation Features

Standardize initial rhythm labels and engineer defibrillation metrics (shock counts and maximum delivered energy) to support downstream survival modeling.


In [ ]:
# Review the heterogeneity of initial rhythm entries before cleaning
df_eda["initial_rhythm"].value_counts(dropna=False)

In [ ]:
import numpy as np

def clean_rhythm(val):
    """Collapse free-text rhythm entries into clinically meaningful buckets."""
    if not isinstance(val, str):
        return np.nan
    v = val.strip().lower()

    if v in ["asystol", "asystole", "asystol.", "asystol", "asysol", "aystol"]:
        return "Asystole"
    if "asy" in v:
        return "Asystole"
    if "pea" in v:
        return "PEA"
    if "vf" in v or "v-fib" in v or "v fib" in v:
        return "VF"
    if "vt" in v or "v-tach" in v or "v tach" in v:
        return "VT"
    if "brady" in v:
        return "Bradycardia"
    if "sinus" in v or v == "sr":
        return "Sinus/Other"
    if v in ["af", "fa"]:
        return "AF"
    if v in ["no", "no cpr", "na", "archive na", "**", "1x 150j", ""]:
        return np.nan
    return "Unknown"

df_eda["initial_rhythm"] = df_eda["initial_rhythm"].apply(clean_rhythm)

In [ ]:
df_eda["initial_rhythm"].value_counts(dropna=False)

The cleaned rhythm categories reveal that asystole dominates the cohort, while shockable rhythms (VF/VT) are rare. A large share of encounters still lack documented rhythm, which should be communicated to clinicians before modeling.


In [ ]:
# Refer to the narrative summary above for interpretation of the cleaned rhythm distribution


In [ ]:
# Examine defibrillation annotations prior to parsing structured metrics
df_eda["defibrillation_info"].value_counts(dropna=False)

In [ ]:
# Parse phrases like "1x 100J" into structured shock_count and max_energy features; default to zero when details are absent

In [ ]:
import re

def extract_shock_values(val):
    """Extract shock count and max energy from defibrillation info."""
    if not isinstance(val, str):
        return pd.Series([0, 0])
    v = val.strip().lower()
    v = re.sub(r'[^0-9xj\s]', ' ', v)
    v = re.sub(r'\s+', ' ', v).strip()
    v = v.replace(' j', 'j').replace('x ', 'x')
    matches = re.findall(r'(\d+)x\s*([0-9]+)j', v)
    if not matches:
        return pd.Series([0, 0])
    shocks = sum(int(a) for a, _ in matches)
    energy = max(int(b) for _, b in matches)
    return pd.Series([shocks, energy])

# Apply the extractor to derive numeric shock features without discarding any rows
df_eda[["shock_count", "max_energy"]] = df_eda["defibrillation_info"].apply(extract_shock_values)


In [ ]:
# Inspect descriptive statistics for engineered defibrillation metrics
df_eda[["max_energy", "shock_count"]].describe()

#### 6.8 Outcome Harmonization

Align ROSC and survival indicators with the same `Y/N/U` semantics and validate cross-field consistency (e.g., ROSC must precede 24-hour survival).


In [ ]:
# Remove the unstructured defibrillation text column after feature extraction
df_eda.drop(columns=["defibrillation_info"], inplace=True, errors="ignore")

In [ ]:
# Review ROSC annotations before normalization
df_eda["rosc"].value_counts(dropna=False)

In [ ]:
df_eda["rosc"] = df_eda["rosc"].str.lower().str.strip()

df_eda["rosc"] = df_eda["rosc"].replace({
    "yes": "Y",
    "no": "N",
    "na": "U"
})

In [ ]:
df_eda["rosc"].value_counts(dropna=False)

In [ ]:
# Validate that ROSC timestamps exist only when ROSC was achieved
invalid_yes = df_eda[(df_eda["rosc"] == "Y") & (df_eda["rosc_dt"].isna())]
invalid_no = df_eda[(df_eda["rosc"] == "N") & (df_eda["rosc_dt"].notna())]

print("ROSC=Y but no time recorded:", len(invalid_yes))
print("ROSC=N but time recorded:", len(invalid_no))

In [ ]:
# Review 24-hour survival outcomes before cleaning
df_eda["survival_24h"].value_counts(dropna=False)


In [ ]:
df_eda["survival_24h"] = df_eda["survival_24h"].str.lower().str.strip()

df_eda["survival_24h"] = df_eda["survival_24h"].replace({
    "yes": "Y",
    "no": "N",
    "na": "U"
})

In [ ]:
# Confirm standardized encoding for 24-hour survival
df_eda["survival_24h"].value_counts(dropna=False)


In [ ]:
df_eda["survival_24h"] = df_eda["survival_24h"].replace("nan", "U")  # Handle textual NaN artifacts
df_eda["survival_24h"] = df_eda["survival_24h"].fillna("U")

In [ ]:
# Sanity check: survival at 24 hours requires ROSC to have occurred
invalid_24h = df_eda[(df_eda["survival_24h"] == "Y") & (df_eda["rosc"] == "N")]
print("Invalid survival_24h (Y but ROSC=N):", len(invalid_24h))

In [ ]:
# Result: no contradictions detected between ROSC status and 24-hour survival

In [ ]:
# Review discharge survival status prior to standardization
df_eda["survival_to_discharge"].value_counts(dropna=False)

In [ ]:
df_eda["survival_to_discharge"] = df_eda["survival_to_discharge"].str.lower().str.strip()
df_eda["survival_to_discharge"] = df_eda["survival_to_discharge"].replace("nan", "U")
df_eda["survival_to_discharge"] = df_eda["survival_to_discharge"].fillna("U")
df_eda["survival_to_discharge"] = df_eda["survival_to_discharge"].replace({
    "yes": "Y",
    "no": "N",
    "na": "U"
})

In [ ]:
df_eda["survival_to_discharge"].value_counts(dropna=False)

#### 6.9 Infer Encounter Origin

Use normalized event locations to backfill missing origin codes (`ND`) while preserving originally captured ICU labels.


In [ ]:
# Standardize origin strings and quantify any discrepancies with normalized event locations
df_eda["origin"] = df_eda["origin"].astype(str).str.strip().str.upper()
df_eda["event_location"] = df_eda["event_location"].astype(str).str.strip().str.upper()

icu_origin_not_icu_event = df_eda[
    (df_eda["origin"] == "ICU") &
    (~df_eda["event_location"].str.contains("ICU"))
]

non_icu_origin_icu_event = df_eda[
    (df_eda["origin"] != "ICU") &
    (df_eda["event_location"].str.contains("ICU"))
]

print("ICU origin but event not in ICU:", len(icu_origin_not_icu_event))
print("Non-ICU origin but event in ICU:", len(non_icu_origin_icu_event))


In [ ]:
# Inspect the event location distribution for cases flagged above
df_eda['event_location'].value_counts(dropna=False)

In [ ]:
# Derive origin categories based on the standardized event location codes


| event_location                                       | Deduce origin as | Rationale                               |
| ---------------------------------------------------- | ---------------- | ---------------------------------------- |
| ICU, POST_ICU, CCU, CVU, NEONATAL_ICU, PEDIATRIC_ICU | ICU              | Intensive or coronary-critical units      |
| MMS, FMS, COVID_WARD                                 | Ward             | General inpatient or specialty wards      |
| PEDIATRIC                                            | Pediatric        | Dedicated pediatric ward                  |
| DSU, OPERATING_ROOM, OBSTETRICS                      | OR/Procedure     | Procedural or peri-operative environments |
| UNKNOWN                                              | Unknown          | No reliable location signal               |


In [ ]:
def deduce_origin(row):
    """Infer a cleaner origin label using event location when the source tag is unknown."""
    origin = str(row["origin"]).strip().upper()
    loc = str(row["event_location"]).strip().upper()

    if origin not in ["ND", "UNKNOWN", "NA", "N/A", ""]:
        return origin

    if loc in ["ICU", "POST_ICU", "CCU", "CVU", "NEONATAL_ICU", "PEDIATRIC_ICU"]:
        return "ICU"
    if loc in ["MMS", "FMS", "COVID_WARD"]:
        return "Ward"
    if loc in ["PEDIATRIC"]:
        return "Pediatric"
    if loc in ["DSU", "OPERATING_ROOM", "OBSTETRICS"]:
        return "OR/Procedure"
    return "Unknown"

df_eda["origin"] = df_eda.apply(deduce_origin, axis=1)


In [ ]:
df_eda["origin"].value_counts(dropna=False)

#### 6.10 Final Quality Checks

- Demographic, comorbidity, event-context, timing, resuscitation, and outcome domains now follow unified encoding schemes.
- Derived timestamps (`cpr_dt`, `rosc_dt`) and shock metrics (`shock_count`, `max_energy`) replace intermediary helper fields.
- Consistency validations (e.g., ROSC vs. survival) return no contradictions, indicating the dataset is ready for modeling or visualization.


**Quality control summary**

- Demographics (`mrn`, `gender`, `birth_year`, `age`) remain internally consistent.
- Comorbidities (CAD, HF, COPD, smoking, etc.) use harmonized categorical codes.
- Event context (`origin`, `event_location`) is normalized with inferred values for `ND` records.
- Timing fields rely on unified datetime/timedelta representations for arrest, CPR, and ROSC.
- Resuscitation metrics (`initial_rhythm`, `shock_count`, `max_energy`) are numerically derived.
- Outcomes (`rosc`, `survival_24h`, `survival_to_discharge`) pass cross-field consistency checks.

In [ ]:
df_eda.info()

#### 7. Export Clean Dataset

Persist the curated dataset to Excel so downstream analysts can consume a single, standardized source.


In [ ]:
df_eda.describe()

In [ ]:
# Export the cleaned DataFrame to an Excel file
df_eda.to_excel("ihca_cleaned.xlsx", index=False)
print("DataFrame exported to ihca_cleaned.xlsx")